In [4]:
def loadDfs():
  dfs_to_load = [
    "~/OneDrive/Mouse2AFC_live/report/aggregates/all_Rbp4_M2.dump",
    #raw_df = pd.read_pickle("~/OneDrive/Mouse2AFC_live/report/aggregates/all_vgat.dump")
  ]
  import pandas as pd
  dfs = []
  for fp in dfs_to_load:
    dfs.append(pd.read_pickle(fp))
  return pd.concat(dfs)
raw_df = loadDfs()
%load_ext autoreload
%autoreload 2
def setup():
  try:
    import analysis
    print("Imported analysis")
  except ImportError:
    pass
    print("Didn't Imported analysis")
  else:
    from pathlib import Path
    cur_dir = Path(analysis.__file__)
    import os
    os.chdir(cur_dir.parent.parent)
    print("Changing directories")
    import sys
    del sys.modules["analysis"]
  from report import analysis
  analysis.setMatplotlibParams(silent=True)
setup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Didn't Imported analysis


ModuleNotFoundError: No module named 'report'

TaskParam.py
__pycache__
analysis.py
analysis_orig.py
batchconvert.py
clr.py
cmds.txt
dates.py
definitions.py
evdaccum
evdaccum.ipynb
mat_reader.py
mat_reader_core.py
mat_to_py.py
matreader
opto
playground.ipynb
plotter.py
psychofit
psychometric.py
py_Mouse2AFC
sessionreport.ipynb
sessionreport.py
stackedperformance.py
states.py
utils.py


In [27]:
from report.evdaccum.reactiontime import reactionTime
from report.evdaccum.evdutils import Kargs
from report.definitions import ExpType

def run():
  short_long_quantile = 0.3
  vs_diff_kargs = Kargs(quantile_top_bottom=0, save_figs=False,
                        exp_type=ExpType.RDK,
                        #save_prefix="proto/x_VsDiff/")
                        save_prefix="proto/ReactionTimeShadlen/")
  import datetime as dt
  # raw_df[(raw_df.Date != dt.date(2021,1,8)) & (raw_df.calcReactionTime < 2)]
  reactionTime(raw_df, min_easiest_perf=75, cut_below_trial_num=30,
              short_long_quantile=short_long_quantile, **vs_diff_kargs)
run()

ImportError: attempted relative import with no known parent package